# load libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
data=pd.read_csv('data/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
data=np.array(data)
m,n=data.shape
np.random.shuffle(data)

# converting data into numpy arrays as it is the data structure best for neural network rather than pandas dataframe

# row and column representation of values

In [33]:
# 1000 data points are set aside for validation
# remaining become features
data_valid=data[0:1000].T
y_valid=data_valid[0]
x_valid=data_valid[1:n]

# training set is remanining data which are transposed
data_train=data[1000:m].T
y_train=data_train[0]
x_train=data_train[1:n]

# 1000 data are set aside for validation purpose as it is important to prevent overfitting so the model can generalized unseen data

# matrix rows need to be transposed for row to columns vice versa

# the rest data is set aside for testing

In [49]:
y_train

array([6, 7, 1, ..., 7, 1, 7], dtype=int64)

# the column x_train has 784 pixels in it

In [52]:
x_train[:,0].shape

(784,)

# neural network

In [78]:
def init_params():
    # initialization of the initial weights
    # valye range not starting with 0 as it slows training time
    w1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1)-0.5

    w2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    
    return w1,b1,w2,b2

# if > than 0 return value else return 0
def ReLU(Z):
    # stable value
    Z=Z-np.max(Z,axis=0)
    exp_Z=np.exp(Z)
    return exp_Z/np.sum(exp_Z,axis=0,keepdims=True)

# sum for each column accross the rows
def softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z))

# method to move forward and update weight and bias value
def forward_prop(w1,b1,w2,b2,X):
    z1=w1.dot(X)+b1
    a1=ReLU(z1)

    z2=w2.dot(a1)+b2
    a2=softmax(z2)
    return z1,a1,a2,z2

# one hot encoding to convert Y labels into a matrix
def one_hot(Y):
    # matrix of 0s, output classes from 0-9 and +1 makes it 10
    one_hot_y=np.zeros((Y.size,Y.max()+1))
    # create an array with range form 0 to number of training examples
    one_hot_y[np.arange(Y.size),Y]=1
    one_hot_y=one_hot_y.T
    return one_hot_y

# derivative of the relu
def derivative_RelU(Z):
    # true values coverted to 1 false to 0
    return Z>0
    
def back_prop(z1,a1,z2,a2,w2,X,Y):
    m=Y.size
    one_hot_y=one_hot(Y)
    dz2=a2-one_hot_y
    dw2=1/m*dz2.dot(a1.T)
    db2=1/m*np.sum(dz2,axis=1,keepdims=True)
    dz1=w2.T.dot(dz2)*derivative_RelU(z1)
    dw1=1/m*dz1.dot(X.T)
    db1=1/m*np.sum(dz1,axis=1,keepdims=True)

    return dw1,db1,dw2,db2

def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
    w1=w1-alpha*dw1
    b1=b1-alpha*db1
    w2=w2-alpha*dw2
    b2=b2-alpha*db2
    return w1,b1,w2,b2

# start with weights value not with 0 as it slows down training time
# ReLU returns value 0 for negative and same value for positive
# softmax is used to bring to scale between 0 and 1 forward propagation ensures that with each step forward values are updated derivative of ReLu tells the slope and how much to adjust values while doing back propagation
# back propgation is done to reduce error value for each iteration, if not decreasing recheck function
# update the weight bias paramters using the alpha value which tells the severity of descent

In [81]:
def get_predictions(a2):
    return np.argmax(a2,0)

def get_accuracy(predictions,Y):
    print(predictions,Y)
    return np.sum(predictions==Y)/Y.size

def gradient_descent(X,Y,iterations,alpha):
    w1,b1,w2,b2=init_params()
    for i in range(iterations):
        z1,a1,z2,a2=forward_prop(w1,b1,w2,b2,X)
        dw1,db1,dw2,db2=back_prop(z1,a1,z2,a2,w2,X,Y)
        w1,b1,w2,b2=update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
        if i%10==0:
            print("Iteration: ",i)
            print("Accuracy: ", get_accuracy(get_predictions(a2),Y))
        return w1,b1,w2,b2

# converting the probability values into actual prediction numbers
# chek how well the model is performaing and able to correctly identify
# initializing the with initial values and running iterations that updates values with forward prop and back prop while updting params for each instance

In [84]:
w1,b1,w2,b2=gradient_descent(x_train,y_train,100,0.1)

C:\Users\atibp\AppData\Local\Temp\ipykernel_8336\808841420.py:18: RuntimeWarning: overflow encountered in exp
  return np.exp(Z)/np.sum(np.exp(Z))
C:\Users\atibp\AppData\Local\Temp\ipykernel_8336\808841420.py:18: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z)/np.sum(np.exp(Z))


AxisError: axis 2 is out of bounds for array of dimension 2